In [1]:
import glob
import sys
import os
import xml.etree.ElementTree as ET
from random import random

In [2]:
DATA_TRAIN_PATH = "/home/jun/Github/Data/Thesis-Data/DragonFruit/DragonFruit-1-VOC/train"

In [12]:
label_path = DATA_TRAIN_PATH + "/labels.txt"

In [13]:
with open(label_path, 'r') as labelfile:
        label_string = ""
        for line in labelfile:
                label_string += line.rstrip()

In [16]:
labels = []
imgnames = []
annotations = {}

In [17]:
labels = label_string.split(',')
labels  = [elem.replace(" ", "") for elem in labels]

In [18]:
labels

['0', '1']

In [19]:
# get image names
for filename in os.listdir(DATA_TRAIN_PATH + "/img"):
    if filename.endswith(".jpg"):
        img = filename.rstrip('.jpg')
        imgnames.append(img)

In [20]:
imgnames

['thanhlong--691-_JPG.rf.32169686197723dc35d1db9d37372cab',
 'thanhlong--732-_JPG.rf.a9368a0fc19e038ed92f5aa5a76c3045',
 'thanhlong--130-_jpg.rf.0202429b730e6d3c47760a31247f556a',
 'thanhlong--782-_JPG.rf.96fc16ab0ee1177d27b26aff20bd260b',
 'thanhlong--110-_jpg.rf.ded32e522a31d20590d57ff7fcd56093',
 'thanhlong--780-_JPG.rf.db4f6df0192f7aa2465687a7721d2807',
 'thanhlong--760-_JPG.rf.068b177f8c84bba73346d292ccc2aea8',
 'thanhlong--117-_jpg.rf.f2a236c6ef8aee43cafc9f8c43798618',
 'thanhlong--441-_jpg.rf.7cbf006b005a52413abc706e19dd8aad',
 'thanhlong--672-_JPG.rf.7267881445e3d4888cedc8af471577a3',
 'thanhlong--157-_jpg.rf.904f93012c76ca0230f8f4ac63ba210c',
 'thanhlong--199-_jpg.rf.f9fb50aa905dd923405aa5c185d36da0',
 'thanhlong--564-_jpg.rf.af37db229c5e79d83919b23ab9b652c8',
 'thanhlong--350-_jpg.rf.c85af42a1e93630471b177d04643c164',
 'thanhlong--282-_jpg.rf.d99a9706a4e96393cc0e193483bd37ac',
 'thanhlong--361-_jpg.rf.75f51a18bf942f20d5b592970d6ec052',
 'thanhlong--69-_jpg.rf.f6ec4e5ca3ae0b52

In [21]:
print("Labels:", labels, "imgcnt:", len(imgnames))

Labels: ['0', '1'] imgcnt: 4138


In [22]:
for label in labels:
        annotations[label] = []

In [23]:
annotations 

{'0': [], '1': []}

In [24]:
for img in imgnames:
    annote = DATA_TRAIN_PATH + "/xml/" + img + '.xml'
    if os.path.isfile(annote):
        tree = ET.parse(annote)
        root = tree.getroot()
        annote_labels = []
        for labelname in root.findall('*/name'):
            labelname = labelname.text
            annote_labels.append(labelname)
            if labelname in labels:
                annotations[labelname].append(img)
        annotations[img] = annote_labels
    else:
        print("Missing annotation for ", annote)
        exit() 

In [25]:
sampler = imgnames.copy()
train_list = []
val_list = []
test_list = []

In [27]:
train = 0.7
val = 0.2
test = 0.1

In [28]:
while len(sampler) > 0:
        dice = random()
        elem = sampler.pop()

        if dice <= test:
            test_list.append(elem)
        elif dice <= (test + val):
            val_list.append(elem)
        else:
            train_list.append(elem) 

In [29]:
print("Training set:", len(train_list), "validation set:", len(val_list), "test set:", len(test_list))

Training set: 2895 validation set: 837 test set: 405


In [31]:
def create_folder(foldername):
    if os.path.exists(foldername):
        print('folder already exists:', foldername)
    else:
        os.makedirs(foldername)

In [32]:
# create the dataset files
create_folder("./ImageSets/Main/")
with open("./ImageSets/Main/train.txt", 'w') as outfile:
    for name in train_list:
        outfile.write(name + "\n")
with open("./ImageSets/Main/val.txt", 'w') as outfile:
    for name in val_list:
        outfile.write(name + "\n")
with open("./ImageSets/Main/trainval.txt", 'w') as outfile:
    for name in train_list:
        outfile.write(name + "\n")
    for name in val_list:
        outfile.write(name + "\n")

In [33]:
# create the individiual files for each label
for label in labels:
    with open("./ImageSets/Main/"+ label +"_train.txt", 'w') as outfile:
        for name in train_list:
            if label in annotations[name]:
                outfile.write(name + " 1\n")
            else:
                outfile.write(name + " -1\n")
    with open("./ImageSets/Main/"+ label +"_val.txt", 'w') as outfile:
        for name in val_list:
            if label in annotations[name]:
                outfile.write(name + " 1\n")
            else:
                outfile.write(name + " -1\n")
    with open("./ImageSets/Main/"+ label +"_test.txt", 'w') as outfile:
        for name in test_list:
            if label in annotations[name]:
                outfile.write(name + " 1\n")
            else:
                outfile.write(name + " -1\n")